# Search-Based Software Engineering Exercise
## Exercise 03 - Test Set Generation with Genetic Programming (Teil 1)
<center>
    
    Johannes Dorn (johannes.dorn@informatik.uni-leipzig.de)
    Max Weber (max.weber@informatik.uni-leipzig.de)
    
    Softwaresysteme - Summer Term 2024
    
</center>
<center>
    <img src='uni-leipzig.png' style="height:5em" />   <img src='SOSY-Logo.png' style="height:5em" />
</center>

# Context
There is an introductory video and a set of slides which should be viewed before continuing with this notebook.

The goal of this exercise is to implement Test-Set Generation with Genetic Programming.

# Setup
We need these libraries

In [1]:
! pip install --user fuzzingbook
! pip install --user numpy
! pip install --user scipy

ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.


## Short Example Class

In [2]:
class BreakMe:
    def __init__(self, x:float):
        if x == 0.0:
            raise ValueError('Computations with zero are dangerous! Please choose another number.')
        self.x = x
        
    def substract(self, y:float):
        self.x = self.x - y 
        return self
    
    def divide(self, y:int):
        self.x = y / self.x
        return self
    
    def get_x(self):
        return self.x

## Execution Coverage

In [3]:
from fuzzingbook.Coverage import Coverage, branch_coverage

In [4]:
def run(method, *args, **kwargs):
    with Coverage() as cov:
        try:
            result = method(*args, **kwargs)
        except:
            result = "FAILED"
    return cov, result

In [5]:
cov, instance = run(BreakMe, 10)
cov.trace()

[('__init__', 3), ('__init__', 5), ('__exit__', 263)]

### Coverage Metrics

#### Statement Coverage

In [6]:
cov.coverage()

{('__exit__', 263), ('__init__', 3), ('__init__', 5)}

#### Branch Coverage

In [7]:
branch_coverage(cov.trace())

{(('__init__', 3), ('__init__', 5)), (('__init__', 5), ('__exit__', 263))}

## Inspection Code

In [8]:
import inspect
def get_methods(class_ref):
    methods = [method for method in dir(class_ref) if not method.startswith("__") or "__init__" in method]
    method_dict = {}
    for method in methods:
        method_dict[method] = {"params": {}}
        method_ref = getattr(class_ref, method)
        m_src = inspect.getsource(method_ref).split("\n")
        method_dict[method]["src"] = m_src
        method_params = inspect.signature(method_ref).parameters
        for param, p_obj in method_params.items():
            if param != "self":
                m_type = p_obj.annotation
                method_dict[method]["params"][param] = m_type  
        method_dict[method]["n_params"] = len(method_dict[method]["params"])
    return method_dict

In [9]:
methods = get_methods(BreakMe)

In [10]:
methods

{'__init__': {'params': {'x': float},
  'src': ['    def __init__(self, x:float):',
   '        if x == 0.0:',
   "            raise ValueError('Computations with zero are dangerous! Please choose another number.')",
   '        self.x = x',
   ''],
  'n_params': 1},
 'divide': {'params': {'y': int},
  'src': ['    def divide(self, y:int):',
   '        self.x = y / self.x',
   '        return self',
   ''],
  'n_params': 1},
 'get_x': {'params': {},
  'src': ['    def get_x(self):', '        return self.x', ''],
  'n_params': 0},
 'substract': {'params': {'y': float},
  'src': ['    def substract(self, y:float):',
   '        self.x = self.x - y ',
   '        return self',
   ''],
  'n_params': 1}}

### Working with Method Inspection data

#### Get data for a specific method

In [11]:
target_method = list(methods.keys())[3]
target_method

'substract'

In [12]:
methods[target_method]

{'params': {'y': float},
 'src': ['    def substract(self, y:float):',
  '        self.x = self.x - y ',
  '        return self',
  ''],
 'n_params': 1}

In [13]:
methods[target_method]["params"]
type(methods[target_method]["params"]['y'])

type

#### run method

In [14]:
method_ref = getattr(instance, target_method)
method_ref(2)

In [15]:
method_ref(2).get_x()

6

In [16]:
erc = 5 # ERC implementation goes here
args = [erc]
run(method_ref, *args)


(<fuzzingbook.Coverage.Coverage at 0x79d400e24f70>,
 <__main__.BreakMe at 0x79d400fd4bb0>)

## Introspection vs Reflection in Python

In [17]:
import random

In [18]:
def run_method_introspec(instance, method, params):
    # your code goes here
    args = []
    for param, p_type in params.items():
        arg = random.randint(2, 7) if p_type is int else random.uniform(1, 4)
        args.append(arg)
    
    method_ref = getattr(instance, method)
    cov, result = run(method_ref, *args)
    print(cov, result)
    

In [19]:
cov, instance = run(BreakMe, 10)

for target_method in list(methods.keys()):
    params = methods[target_method]["params"]
    run_method_introspec(instance, target_method, params)

<fuzzingbook.Coverage.Coverage object at 0x79d400e26890> None
<fuzzingbook.Coverage.Coverage object at 0x79d400e26890> <__main__.BreakMe object at 0x79d400e26d10>
<fuzzingbook.Coverage.Coverage object at 0x79d400e26890> 1.4496454984745704
<fuzzingbook.Coverage.Coverage object at 0x79d400e26890> <__main__.BreakMe object at 0x79d400e26d10>
